In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import multiprocessing
from keras import backend as K
import joblib
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score,auc , recall_score, accuracy_score, confusion_matrix
import torch.nn as nn
from torch import optim
import csv
import torch
from sklearn.feature_selection import SelectPercentile, f_classif
from tqdm import tqdm
from model import DNN2
import argparse
from sklearn.metrics import f1_score
from data_tool import label_encode,label_decode,eachAccu
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import os
from dataloader import dataSet,data_prefetcher
from sklearn.metrics import f1_score

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
NUMWORKER = 4

In [ ]:
seed =999
fea_dim = 2368 # 512/2048/2368/832
fea_type = 'tfidf'
X = joblib.load('./fea_sel/{}/{}_train-{}.pkl'.format(fea_type,fea_type.lower(),fea_dim))
hashtag = [] 

label = joblib.load('./fea_sel/label/label8.pkl')
label_encoder = LabelEncoder()
label_encoder.fit(label)
y = label_encode(label_encoder,label)
NUM_WORKER = 4
lab_size = 8
max_features = 2048
MAXEPOCH = 15
Test =False

layer_sizes = [fea_dim,int(fea_dim/4), int(fea_dim/8),int(fea_dim/32)]
BZ= [512]
LR = [0.001]
DP = [0,0.2]
kf = KFold(n_splits=5,shuffle=True,random_state=2)

In [ ]:
def train(model_clf, trainLoader, optimizer,device=DEVICE):
    print(torch.torch.cuda.current_device())
#    num =torch.cuda.current_device()
#    print(torch.cuda.get_device_name(num))
    
    num_iter = len(trainLoader)
    prefetcher = data_prefetcher(trainLoader)
    model_clf = model_clf.train()
    preds = []
    gnds = []
    losses = []
#    for  x_batch, y_batch,ID in tqdm(trainLoader):
    for batch_idx in tqdm(range(num_iter)):
        x_batch,y_batch,ID = prefetcher.next()
        
        x_batch = x_batch.to(DEVICE).squeeze(1).float()
        y_batch = y_batch.flatten().to(DEVICE)
#        print(x_batch.is_cuda)
#        print(y_batch.is_cuda) 
        
        optimizer.zero_grad()
        
        out_train = model_clf(x_batch)
        lossCrossEntropyTrain = loss_CrossEntropy(out_train, y_batch)
        losses.append(lossCrossEntropyTrain.item())
        preds.extend(np.argmax(out_train.detach().cpu().numpy(), axis=1).tolist())
        gnds.extend(list(y_batch.detach().cpu().numpy().astype('int')))

        lossCrossEntropyTrain.backward()
        optimizer.step()
        
#        print('\ntrain loss:{:.5}'.format(lossCrossEntropyTrain.item()))
    losses = np.array(losses).mean()
    rint('\ntest loss:{:.5}'.format(losses))
    return model_clf, optimizer, preds, gnds, losses


def test(model_clf, testLoader,device=DEVICE):
    model_clf = model_clf.eval()
    IDs = []
    preds = []
    gnds = []
    losses = []
    outList = []
    num_iter = len(trainLoader)
    prefetcher = data_prefetcher(trainLoader)
#    for  x_batch, y_batch,ID in tqdm(testLoader):
    for batch_idx in tqdm(range(num_iter)):
        x_batch,y_batch,ID = prefetcher.next()
        # forward
        x_batch = x_batch.to(DEVICE).squeeze(1).float()
        y_batch = y_batch.flatten().to(DEVICE)
        
        out_test = model_clf(x_batch )
        lossCrossEntropyTest = loss_CrossEntropy(out_test, y_batch)
        losses.append(lossCrossEntropyTest.item())
        
        predictTest = np.argmax(out_test.detach().cpu().numpy(), axis=1)
        out = list(out_test.detach().cpu().numpy())
        outList.extend(list(map(lambda x: x[1], out)))
        preds.extend(predictTest.tolist())
        gnds.extend(list(y_batch.detach().cpu().numpy().astype('int')))
    losses = np.array(losses).mean()
    print('\ntest loss:{:.5}'.format(losses))
    return model_clf,outList, preds, gnds , losses
def writer_csv(logPath, logging):
    f = open(logPath,'a')
    w = csv.writer(f,lineterminator = '\r')
    w.writerow(logging)
    f.close() 

In [ ]:
for bz in BZ:
            for lr1 in LR:
                for dropout_p in DP:
                    X_idex = np.arange(1455563).reshape(-1,1)
                    for train_index, test_index in kf.split(X_idex):
                        dataset = dataSet(fea_type,str(fea_dim),train_index,train=True)
                        trainLoader = torch.utils.data.DataLoader(dataset,batch_size =bz,shuffle=True,num_workers=NUM_WORKER,pin_memory=True)
                        dataset = dataSet(fea_type,str(fea_dim),test_index,train=True)
                        testLoader = torch.utils.data.DataLoader(dataset,batch_size =bz,shuffle=True,num_workers=NUM_WORKER,pin_memory=True)
                        del dataset
                        
                        lossTrain, lossTest = [[] for i in range(MAXEPOCH)],[[] for i in range(MAXEPOCH)]
                        for epoch in range(MAXEPOCH):
    #                        print(epoch)
                            predTestAll,gndTestAll,outAll = [],[],[]
                            predTrainAll,gndTrainAll = [],[] 
                            if epoch==0:
                            
                                model_clf = DNN2(layer_sizes=layer_sizes, output_size=lab_size , drop_p=dropout_p).to(DEVICE)
                                loss_CrossEntropy = nn.CrossEntropyLoss()
                                optimizer = optim.Adam(model_clf.parameters(), lr=lr1)
                                # =============================================================================
                                #                    Training & Testing & saving models
                                # =============================================================================
            #                    print('\n=== training lr{} dp{}  cvIdx{} epo{}===\n'.format(lr1,dropout_p,cvIdx,epoch))
                                print('\n=== training lr{} dp{}   epo{}===\n'.format(lr1,dropout_p,epoch))
                                model_clf, optimizer, predTrain, gndTrain, losses = train(model_clf, trainLoader, optimizer)
                                lossTrain[epoch].append(losses)
                                predTrainAll.extend(predTrain)
                                gndTrainAll.extend(gndTrain)   
                                
                                # Testing   
                #                print('\n=== testing lr{} dp{}  cvIdx{} epo{} ===\n'.format(lr1,dropout_p,cvIdx,epoch))
                                print('\n=== testing lr{} dp{}  ===\n'.format(lr1,dropout_p))
                                model_clf, outTest,predTest, gndTest, losses = test(model_clf, testLoader)
                                lossTest[epoch].append(losses)
                                predTestAll.extend(predTest)
                                gndTestAll.extend(gndTest)
                                outAll.extend(outTest)
                                # =============================================================================
                                #                       Evaluate
                                # =============================================================================
                                uarTrain = recall_score(gndTrainAll, predTrainAll, average='macro')
                                accuTrain = accuracy_score(gndTrainAll, predTrainAll)
                                uarTest = recall_score(gndTestAll, predTestAll, average='macro')
            #                    aucTest = 0
                                _ = eachAccu(predTestAll, gndTestAll) 
                                accuTest = accuracy_score(gndTestAll, predTestAll)
                                cmTest = confusion_matrix(gndTestAll, predTestAll)
                        '''if Test:
                            logg = './submission.csv'
                            writer_csv(logg, ['id','emotion'])
                            for id_,emo in zip(testID,pred):
                                if emo == 0: w = 'sadness'
                                elif emo == 1: w = 'disgust'
                                elif emo == 2: w = 'anticipation'
                                elif emo == 3: w = 'joy'
                                elif emo == 4: w = 'trust'
                                elif emo == 5: w = 'anger'
                                elif emo == 6: w = 'fear'
                                elif emo == 7: w = 'surprise'
                                ww= [id_,w]
                                writer_csv(logg, ww)'''
                        # =============================================================================
                        #                       Plotting
                        # =============================================================================
                        for i in range(MAXEPOCH):
                            lossTrain[i] = np.array(lossTrain[i]).mean(axis = 0)
                            lossTest[i] = np.array(lossTest[i]).mean(axis = 0)
                            
                        lossTrain = np.asarray(lossTrain).T
                        lossTest = np.asarray(lossTest).T
                        
                        plt.figure(figsize=(8,4))
                        
                        plt.plot(lossTrain,label="$Training loss$",color="blue",linewidth=1)
                        plt.plot(lossTest,label="$Testing loss$",color='red',linewidth=2)
                        plt.xlabel("Epoch")
                        plt.ylabel("Loss")
                        plt.xticks(range(len(lossTrain)))
                        plt.title("Loss Plot")
                        plt.legend()
                        plt.show()